In [1]:
from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb

import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = DefaultArgsNamespace()

train_loader, val_loader, test_loader = eee_get_dataloaders(args)

# # get input feature  size
# dataiter = next(iter(train_loader))
# batch =  dataiter


# # Access the parsed arguments
model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

dummy_input = torch.randn(1, 30, 1024) # batch, num_frames, channels, height, width

dummy_input = dummy_input.to(args.device)

dummy_output = model(dummy_input)   

print(dummy_output.shape)




train dataset size:  5686
val dataset size:  1892
test dataset size:  2477


/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 16])


In [2]:
def preprocess(x, modality, backbone):
    # check the shape of the input tensor
    # print(x)
    feature = None
    label = x['keystep_id']
    if('video' in modality):
        feature = None
        x = x['frames']
        # extract resnet50 features
        x = x.to(device)
        x = backbone(x)
        feature = x
    

    elif ( 'flow' in modality and  'rgb' in modality):

        # I3D features are already extracted
        flow = x['flow'].float()
        rgb = x['rgb'].float()
        feature = torch.cat((flow, rgb), dim=-1).float()

    elif ('rgb' in modality):
        # I3D features are already extracted
        feature = x['rgb'].float()

    elif ('flow' in modality):
        # I3D features are already extracted
        feature = x['flow'].float()

    elif ('audio' in modality):
        # Audio features are already extracted
        feature = x['audio'].float()

    feature_size = feature.shape[-1]

    if(feature is not None):
        feature = feature.to(device)
        label = label.to(device)
    return feature, feature_size, label

In [3]:
keysteps = args.dataloader_params['keysteps']
out_classes = len(keysteps)
print(out_classes)

31


In [4]:
modality = args.dataloader_params['modality']
print("Modality: ", modality)

# Find feature dimension
input, feature_size, label = preprocess(next(iter(train_loader)), args.dataloader_params['modality'], model.extract_resnet)
print("Feature size: ", feature_size)

args.transformer_params['input_dim'] = feature_size
args.transformer_params['output_dim'] = out_classes

model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

wandb_logger = wandb.init(
    # set the wandb project where this run will be logged
    project="EgoExoEMS",
    group="Keystep Recognition",
    mode="disabled",
    name="Testing on EgoExoEMS with I3D RGB Features - ICRA Model",
    notes="initial attempt ICRA model with I3D RGB features",
    config={
    "args": args,
    }
)



Modality:  ['video']


Feature size:  2048


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [6]:
# # print one batch
# for i, batch in enumerate(train_loader):
#     input,feature_size, label = preprocess(batch, args.dataloader_params['modality'], model.extract_resnet)
#     print("Model input:",input.shape)
#     output = model(input)
#     print("Model output:",output.shape)
#     print("Label:",label)
#     break

results_dir = './results'
# # # Train the model
for epoch in range(args.learning_params["epochs"]):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, wandb_logger, modality=modality)
    val_loss = validate(model, val_loader, criterion, device, wandb_logger, modality=modality)
    scheduler.step()
    print(f"Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")
    results = test_model(model, test_loader, criterion, device, wandb_logger, modality=modality, epoch=epoch, results_dir=results_dir)
    print(results)
    break


Pred: tensor([25], device='cuda:0') GT: tensor([4], device='cuda:0')
Batch: 0, Loss: 4.087499618530273
Epoch: 0, Train Loss: 0.0007188708439202029, Val Loss: 0.0018713998492121949
0.0


/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
